In [ ]:
# Import modules
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import data
path = '/content/drive/MyDrive/Masterthesis/data/patents.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
categories = df['category'].copy()

# Synthesis
categories = categories.str.replace('1.1.1.1', '1.1.1')
categories = categories.str.replace('1.1.1.2', '1.1.2')
categories = categories.str.replace('1.1.1.3', '1.1.3')
categories = categories.str.replace('1.1.1.4', '1.1.4')
categories = categories.str.replace('1.1.2.1', '1.1.5')
categories = categories.str.replace('1.1.2.2', '1.1.6')
categories = categories.str.replace('1.1.2.3', '1.1.7')
categories = categories.str.replace('1.1.2.4', '1.1.8')
categories = categories.str.replace('1.1.3', '1.1.9')

# Applications
applications = ~categories.str.startswith('1')
categories[applications] = '2.' + categories[applications]

# Photo electrics
categories = categories.str.replace('2.5. ', '2.5.3 ')

df['category'] = categories
df.head()

## Data balance
In this section we analyse the balance of the dataset when removing patents.

### Adjust abstract lenght
We start by plotting a histogram of the different abstract lenghts

In [ ]:
# Set NaN rows to length 0
df['abs_len'] = df['abstract'].str.len().fillna(0)

# Plots histrogram of lenghts
df['abs_len'].plot.hist(bins = 100, title = 'histogram of abstract lenghts');

Removing documents without short abstracts and short descriptions

In [ ]:
# Remove abstracts shorter than 100 characters
abs_len_df = df[df['abstract'].str.len() > 100]

# Plots histrogram of lenghts
abs_len_df['abs_len'].plot.hist('abs_len', bins = 100, title = 'Histogram of abstract lengths after short abstract removal');

Adjust description lenght

In [ ]:
# Set NaN rows to length 0
df['desc_len'] = df['description'].str.len().fillna(0)

# Plots histrogram of lenghts
df['desc_len'].plot.hist('desc_len', bins = 100, title = 'histogram of description lenghts');

In [ ]:
# Set NaN rows to length 0
abs_len_df['desc_len'] = abs_len_df['description'].str.len().fillna(0)

# Remove descriptions shorter than 100 character
desc_abs_len_df = abs_len_df[abs_len_df['description'].str.len() > 100]

# Plots histrogram of lenghts
desc_abs_len_df['desc_len'].plot.hist( bins = 200, title = 'Histogram of description lengths after short abstract and short description removal');

## Number of patents removed
How many patents did we remove in total?

In [ ]:
# Print the amount of patents we have
print("Number of original patents:      ", len(df))
print("Number of patents after removal: ", len(desc_abs_len_df))
print("Number of removed patents:       ", len(df) - len(desc_abs_len_df))

## Country balance
How many patents do we have from each country after removing short patents

In [ ]:
# Get column describing country
df['country'] = df['id'].str[:2]
desc_abs_len_df['country'] = desc_abs_len_df['id'].str[:2]

In [ ]:
# Plots count of column
def plot_occurance(df,column, title = None):
  counts =  df.groupby(column).count().iloc[:,0]
  counts.sort_values(ascending=False).plot(kind='bar', title = title);

In [ ]:
plot_occurance(df, 'country', 'Original country occurance')

In [ ]:
plot_occurance(desc_abs_len_df, 'country', title = 'Country occurance after short abstract removal')

# Analyse class balance
How many patents do we have of each over arching class after removals

In [ ]:
# Set category to first character of category column for both the original and removed dataframes
df['first_category'] = df['category'].str[:1]
desc_abs_len_df['first_category'] = desc_abs_len_df['category'].str[:1]

In [ ]:
plot_occurance(df, 'first_category', title = 'Original category balance')

In [ ]:
plot_occurance(desc_abs_len_df, 'first_category', title = 'Category balance after short abstract removal')

In [ ]:
desc_abs_len_df['category'].value_counts().sort_index()

# Saving destilled dataset

In [ ]:
desc_abs_len_df.to_csv('/content/drive/MyDrive/Masterthesis/data/destilled_patents.csv', index = None)